In [1]:
from logging import exception
#@title Step -1: Mount drive
#@markdown Run this cell. If prompted, press "Connect to Google Drive" and select your Google account.
#@markdown Then, under the folder icon 📁 on the left panel, you should see the folder **drive** appear.
from google.colab import drive
from IPython.display import display, Markdown, HTML
import os, sys

%load_ext autoreload
%autoreload 2
try:
  drive.mount('/content/drive', force_remount=False)
  # os.chdir('/content/drive/My Drive/DLE-Feb23/Projects')
  sys.path.append('/content/drive/MyDrive/DLE-Jun23/Projects')
  os.chdir('/content/drive/MyDrive/Colab Notebooks/')
  display("⭐ Mounted successfully!")
except:
  display(HTML('<span style="color:red">An error occurred. Try again!</span>'))


Mounted at /content/drive


'⭐ Mounted successfully!'

In [2]:
#@title Step 0: Import Packages
%%capture
!pip install openai
!pip install scikit-learn
!pip install matplotlib
!pip install transformers

import json
import pandas as pd
import openai
import numpy as np
from tqdm import tqdm
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from dle_utils.dle_utils import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import transformers
import ast
import pprint

## Step 1: Accessing Open AI's API

Before starting this assignment, you'll need to do the following steps:

1. [Create an account on OpenAI](https://auth0.openai.com/u/signup/identifier?state=hKFo2SBzQm1sd2pOTUJ2SG9sSHBBdUU1bGNpdllveFhrWW8wc6Fur3VuaXZlcnNhbC1sb2dpbqN0aWTZIHctYmtFZ3h0ck9saDNVWFhUTnRmckM5azd0RjUtMjMwo2NpZNkgRFJpdnNubTJNdTQyVDNLT3BxZHR3QjNOWXZpSFl6d0Q) (if you have not done so already).

2. In the upper right hand corner, click on your profile icon and select  **View API keys**. On the API keys page, copy your API key into the cell below.

<img src="https://drive.google.com/uc?id=1ctYY7b1TqauMobC6J-Vm_VGBF9nv_4XI" width=800/>

You will only see this key once, so make sure to copy it in a safe place!

3. On the account dropdown, go to **Manage account** to make sure you have access to credits in your account. If you started a free trial, you should have $5.00. If you're like me and your credits have expired/run out, you may want to create a new account! (Otherwise, ChatGPT API access is quite cheap, so you may want to consider adding payment information)

**Remember to remove your API key from the homework before submitting it!**


In [3]:
# Put down your API key here

openai.api_key = "YOUR_API_KEY"

## Deep learning APIs

In a not so distance past, loading and using deep learning models was a laborious and difficult task, involving lots of helper functions and niche knowledge about how models worked.

Now, thanks to the democratization of AI, the most powerful models are available through APIs. In this assignment, we'll give a quick tutorial on the most recent and exciting API: Open AI's ChatGPT API. Then, we'll go through a real-world example on how they might be used to help you in your own work.

### Using OpenAI's API

OpenAI's API is really accessible via Python. There are two basic modes in this API.

1. ```openai.Embedding```
2. ```openai.ChatCompletion```

Roughly speaking, these two modes are just the two levels of abstraction. At the first level, the Embedding API gives you the most raw form of the model -- the embedding vectors of a given text. Next, the ChatCompletion API is a higher level version of the embedding API, where the model has been trained to interpret instructions. (By the way, there are other modules like [Audio](https://platform.openai.com/docs/guides/speech-to-text) which are really cool, but we wont' cover here!)

## Embeddings

The first use case is in extracting model embeddings. This is identical to the model embeddings used in week 2's assignment. Here, we'll just do a quick demo in terms of using these embeddings for a downstream task.

In the cell below, load ```local_df```. In this, you'll see 1000 genuine Google Local reviews from places in Washington, D.C. Each review is associated with a rating from 1-5.

In [4]:
local_df = pd.read_csv('/content/drive/MyDrive/DLE-Jun23/Projects/dle_utils/data/google_local_data.csv', index_col=0)
local_df.head()

,rating,text
0,4,"the food is really good, I had a great experie..."
1,4,High prices for not so big portions! But I thi...
2,2,"The gelato looked ""old"" and ended up having co..."
3,4,(Translated by Google) The fast food restauran...
4,1,(Translated by Google) The food is barely edib...


Let's say that Google users have a tendency to forget to put a 1-5 star rating next to their review. Can we use the embedding API to predict which rating should be associated with each review?

### Extracting embeddings

Use ```openai.Embedding.create()``` to extract embeddings for each review in ```local_df```.  ```openai.Embedding.create()``` takes in two parameters: ```model``` and ```input```. (You can read embeddings [here](https://platform.openai.com/docs/guides/embeddings/what-are-embeddings)).

For ```model```, use ```"text-embedding-ada-002"```, which is the best, fastest, and cheapest ($0.0004 / 1k tokens).

For ```input```, you just need to enter the text you want to embed as a string.

In the cell below, take the first review from ```local_df``` and extract its embedding.

(Hint: For a Pandas dataframe, you can access the text from a specific row with ```local_df.iloc[i]['text']```, where ```i``` is the index of the row.

In [5]:
def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return openai.Embedding.create(input = [text], model=model)

In [6]:
# Populate response with the embedding of the first row
sample_review = local_df.iloc[0]['text']

# Extract embedding
response = get_embedding(sample_review, model="text-embedding-ada-002")

print(response.keys())

dict_keys(['object', 'data', 'model', 'usage'])


In [7]:
check('4.1.1', response)

## Correct! 🎉

The output, ```response```, is accessible as a dictionary.

Find where the embeddings are stored in this dictionary. (Hint: You can get the keys to a dictionary with ```.keys()```). The embeddings will be a list of lists, which we want to convert to a numpy array.

In [8]:
# Extract the embeddings from the response and convert to a numpy array

emb_arr = response['data'][0]['embedding']
emb_arr_type = type(emb_arr)
emb_arr = np.array(emb_arr)



In [9]:
check('4.1.2', emb_arr)

## Correct! 🎉

Great! Now that we can extract the embedding for any piece of text, we want to get the embeddings for a subset of ```local_df```.

You'll notice the shape of ```emb_arr``` is ```(1536,)```, which is the number of dimensions each embedding is.

In the cell below, populate the list ```sample_emb``` with the embeddings for the first **20** reviews. Then, convert ```sample_emb``` into a numpy array. The shape of ```sample_emb``` should be ```(20, 1536)```.

This should not take more than a couple of minutes.

(You may get a notice like ```RateLimitError: The server is currently overloaded with other requests. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists.``` If this happens, you may need to wait a few more minutes. It may also help to use a [sleep](https://www.programiz.com/python-programming/time/sleep) timer to keep under the rate limit.)

In [10]:
def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   response=openai.Embedding.create(input = [text], model=model)
   return response['data'][0]['embedding']

sample_df = local_df.iloc[0:20,1]
embedding_df = sample_df.apply(lambda x: get_embedding(x))


In [11]:
embedding_df = embedding_df.apply(lambda x: np.array(x).reshape(-1,1536))

In [12]:
embedding_arrays = embedding_df.values
arr = np.array([])
for i in embedding_arrays:
  arr=np.append(arr,i)

In [13]:
sample_emb = arr.reshape(20,-1)

In [14]:
check('4.1.3', sample_emb)

## Correct! 🎉

Great! Next, we want to gather emebeddings for all 1000 reviews. If you have an OpenAI API account that is older than 48 hours old and you have your credit card attached, you can query 1000 reviews easily. However, newer accounts are rate limited. If you find yourself rate-limited, just skip past this next part and we'll provide you the embeddings for the full 1000 as a ```.npy``` file.

<img src="https://drive.google.com/uc?id=1exdv5DIvWvw9-0O8xYazttEyiuFcdzDg" width=250/>.

In [15]:
# Extract embeddings for all reviews in local_df
# Populate local_emb with a numpy array of embeddings
# Do this part if you aren't rate-limited!

def get_embedding(input_string):
  ...

#local_emb = ...

In [16]:
# If you are rate-limited, just run this!

local_emb = np.load('/content/drive/MyDrive/DLE-Jun23/Projects/dle_utils/data/local_emb.npy')

Now that we have a dataset of 1000 embeddings, let's see if we can train a machine learning model to predict the ratings. We can start with doing a binary prediction of whether the review was a low rating (1, 2, or 3) or a high rating (4 or 5).

The cell below has been given to you to train a basic model on this binary task.

In [17]:
# Just run this!
y = (local_df['rating'] > 3).astype(int).to_list()
X = local_emb

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(max_iter=1000, random_state=42).fit(X_train, y_train)

print(f'Model AUC: {roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])}')

Model AUC: 0.9505813953488372


Not bad, right? An AUC of 0.95 is pretty impressive.

Even though OpenAI's API is pretty cheap, using it across a lot of queries could still cost us some serious dough. Fortunately, free versions of large language models exist. Even better news is that they are incredibly easy to use. Run the line below to load up one of these models.

In [18]:
from transformers import pipeline
pipeline = pipeline('feature-extraction', model='xlnet-base-cased')

In [19]:
pipeline

The function ```pipeline``` takes in a string and returns the embedding. However, it will give one embedding for each token, so you'll have to take the average over all tokens like you did in the week 2 assignment.

One advantage of using Huggingface is that it's totally free -- no rate limits :)

In the cell below, extract all the embeddings using ```pipeline``` and store the embeddings as a numpy array in ```local_emb_hf``` (hf stands for huggingface). After doing so, run the machine learning model and see how well these embeddings perform on this task. (It might take a couple minutes!)


In [19]:
# Tokenize sentence and extract embedding from the trained layer
import torch
def sent_embedding(sentence):
    embeddings = torch.tensor(pipeline(sentence))

    sentence_emb = torch.mean(embeddings, dim=1, keepdim=True)
    # Compute the mean of the layer across all words in the sentence.
    mean_emb = torch.squeeze(sentence_emb).reshape(1,-1)
    return np.array(mean_emb)

In [20]:
sent_embedding(local_df.iloc[0]['text']).shape

(1, 768)

In [21]:
# Extract embeddings for all reviews in local_df
# Populate local_emb_hf with a numpy array of embeddings
local_emb_hf = np.array([])
for i in range(local_df.shape[0]):
  sent =local_df.iloc[i]['text']
  embedd = sent_embedding(sent)
  local_emb_hf=np.append(local_emb_hf,embedd)


In [22]:
local_emb_hf = local_emb_hf.reshape(local_df.shape[0],-1)

In [23]:
local_emb_hf.shape

(1000, 768)

In [24]:
check('4.1.4', local_emb_hf)

## Correct! 🎉

In [25]:
y = (local_df['rating'] > 3).astype(int).to_list()
X = local_emb_hf

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(max_iter=1000, random_state=42).fit(X_train, y_train)

print(f'Model AUC: {roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])}')

Model AUC: 0.8898847975882859


If you did this right, you should be seeing an AUC of around 0.89. It's not bad, but certaintly not as good as 0.95. As you can see, performance comes with a price.

If you'd like, you can try a couple other models to see how well they do. [List of huggingface models](https://huggingface.co/models).

**Challenge question: Can you find a huggingface model that performs at least 0.90 AUC?**

At the end of the day, whether you use OpenAI's API or a free model from huggingface, the choice just depends on what task you need them for.

___

But what about tasks which we don't have a label for? Suppose we wanted to get the predicted reviews, but we don't have training data? A few years ago, we would have been stuck trying to gather more data. However, now with OpenAI's chat completion API, we may be able to do more than we think.

## Chat Completion

Chat completion provides a whole new dimension of functionality on large language models. Whereas embeddings are hard to interpret machine language, the chat completion API allows us to give the model instructions on how to interpret the prompt. To illustrate this, we will be working with real reviews of restaurants from Google. Run the cell below to read in the data.

In [26]:
review_df = pd.read_csv('/content/drive/MyDrive/DLE-Jun23/Projects/dle_utils/data/review_df_cols.csv', index_col=[0])
review_df.head()

,business_id,review_text,rating
0,60567465d335d0abfb415b26,The tang of the tomato sauce is outstanding. A...,4
1,6050fa9f5b4ccec8d5cae994,Chicken and waffles were really good!,5
2,604be10877e81aaed3cc9a1e,The appetizer of colossal shrimp was very good...,4
3,60411e017cd8bf130362365a,The fish tacos here omg! The salad was great ...,5
4,604139dd7cd8bf1303624208,"Ribs are great, as are the mac and cheese, fri...",4


You'll notice three columns: ```business_id, review_text, rating```. The business is a unique id associated with each restaurant. The rating is a score from 1-5, and the review_text is, well, the review text.

Rather than try to predict ratings, we're going to try something a little more complicated. Imagine you're working for a restaurant recommendation company, and this user feedback comes in. Your manager wants you to solve this problem and you're tasked with finding the answer using deep learning:

**I love using your app to find restaurants, but each time I get to a restaurant, I don't know what to order. I know that other users write reviews about specific menu items, but it's a time-consuming process to go through each review to find them. I wish there was a quick summary of the reviews for each menu item!**

Let's break down this problem into a couple steps:

1. We want to collect all the reviews for a specific restaurant. For example, we can look at the restaurant with ```business_id=605618c0d335d0abfb415a0f```. Run the cell below to get a sample of the reviews:

In [27]:
# Just run this!
reviews = review_df[review_df['business_id']=='605618c0d335d0abfb415a0f']['review_text'].to_list()
print('\n'.join(reviews))

I had the elk burger and my wife had the hot roast beef sandwich, I think the hot sandwich is better tasting.
Portions are quite large, you get A WHOLE LOAF OF BREAD (see photo). Pie was great, but they ran out of banana blueberry cream pie so I didnt get try Ethels famous cream topping and ask for the family recipe.
I got the cheeseburger with bacon and my husband and mom and sister got the Hawaiian burger! They are big burgers so I recommend sharing!
The fries are tasty and perfectly crunchy, the burgers and salad are great too.
The fried green beans appetizer was good. The elk burger and Reuben were average. The boysenberry tasted good with the vanilla ice cream though.
The Rodeo Burger is DELICIOUS!! But man, that burger... We couldn't make up our minds on either the Apple Pie or the Chocolate Pie.
I had the French dip that fantastic, REAL au jus, hubby had chicken fried steak that he loved, my son had a BLT on sourdough and daughter had the fish and chips.
I got the elk burger wit

2. For each review, we want to ask ChatGPT to extract the reviews for specific menu items. There are a few formats we can get this in, but to make it clear, we can have ChatGPT return it as a json/dictionary.

The idea is to get a lookup table of items, plus their reviews. For example,

```{'chow mein': 'Most users found it delicious, with some thinking it is too spicy', \
    'orange chicken': 'This is a crowd favorite, and has been described as "tangy" and "zesty"'}```

Let's start with one of the reviews:

In [28]:
# Just run this!
sample_review = reviews[8]
print(sample_review)

I ordered a local beer and a hamburger, but it was okay. The regular burger was better than the sandwich. The Turkish sandwich had a unique jam inside.


Now, we want to get ChatGPT to convert this review to a dictionary:

To query OpenAI's ChatGPT API, use ```openai.ChatCompletion.create()```. You need to pass two parameters:

1. ```model```, which should be 'gpt-3.5-turbo'
2. ```messages```, which should be a list of dictionaries. Each dictionary should contain two entries: ```'role'``` and ```'content'```.

```'role'``` can either be ```'system'```, ```'user'```, or ```'assistant'```.

The role ```'system'``` is meant to be a persistent instruction for the prompt. You can write something like ```'You are an restaurant review writer whose job it is to help people find good menu items. You will return responses is json dictionary format only'```.

The role ```'user'``` is the prompt given. For example, the prompt can be: ```'The following text is a list of restaurant reviews. For each unique menu item mentioned, give a summary of the user reviews for that menu item. Return the output in the form of a json dictionary, where the keys are unique menu items and the values are the summary.'```


In [29]:
# Run these lines
system_prompt = 'You are an restaurant review writer whose job it is to help \
               people find good menu items. You will return responses is json \
                dictionary format only'

# Feel free to modify this!
prompt = f'The following text is a list of restaurant reviews. \
        For each unique menu item mentioned, give a summary of \
        the user reviews for that menu item. \
        Return the output in the form of a json dictionary, \
        where the keys are unique menu items \
        and the values are the summary. : "{sample_review}"'


The ```.create``` function expects the parameter ```messages``` to be a list, with each element as a dictionary which specifies who (```role```) and what (```content```) is said.

So for example, if we wanted to ask ChatGPT what the first US state is, we could pass in the following into ```messages```:

```messages=[{'role': 'system', 'content': 'You are a helpful tour guide.'}, {'role': 'user', 'content': 'What is the first US state?'}]```

In the cell below, pass the example about US states into ```openai.ChatCompletion.create``` and print out what the output is in ```response```:

In [30]:
messages=[{'role': 'system', 'content': 'You are a helpful tour guide.'}, {'role': 'user', 'content': 'What is the first US state?'}]

In [31]:
response = openai.ChatCompletion.create(model="gpt-3.5-turbo",messages=messages)

print(response)

{
  "id": "chatcmpl-7hdapDmW3Nu8oCUxm11uwERqHYzIx",
  "object": "chat.completion",
  "created": 1690634655,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The first state to join the United States was Delaware. It became a state on December 7, 1787, making it the first state to ratify the U.S. Constitution."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 25,
    "completion_tokens": 38,
    "total_tokens": 63
  }
}


As you see, the response itself is a dictionary. Here are a couple key things to point out about this response:

1. The actual response is found in "content" under "choices".
2. The "usage" dictionary may be helpful for understanding how many credits you have used per query (ie. for logging purposes). But don't worry about that for this project.

With this information, we are ready to get ChatGPT to convert a restaurant review into a menu item-specific look-up table.

In the cell below, query the API given the variables ```prompt``` and ```system_prompt``` defined for above restaurant reviews:

In [33]:
messages = messages=[{'role': 'system', 'content': 'You are a helpful tour guide.'}, {'role': 'user', 'content': 'What is the first US state?'}]
response = openai.ChatCompletion.create(model="gpt-3.5-turbo",messages=messages)
response_as_dict =response['choices'][0]["message"]["content"]

In [34]:
messages=[{'role': 'system', 'content': f"{system_prompt}"}, {'role': 'user', 'content': f"{prompt}"}]

In [35]:
messages=[{'role': 'system', 'content': f"{system_prompt}"}, {'role': 'user', 'content': f"{prompt}"}]
response = openai.ChatCompletion.create(model="gpt-3.5-turbo",messages=messages)

print(response)

{
  "id": "chatcmpl-7cJ1GmuGOCJBexielRKYxoLq9dkmp",
  "object": "chat.completion",
  "created": 1689363930,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "{\n   \"local beer\": \"The user mentioned that they ordered a local beer, but did not provide any further details about it.\",\n   \"hamburger\": \"The user mentioned that they ordered a hamburger and thought it was okay.\",\n   \"regular burger\": \"The user mentioned that they had a regular burger and liked it better than the sandwich.\",\n   \"sandwich\": \"The user mentioned that they had a sandwich with a unique jam inside, but did not provide any further details about it.\",\n   \"Turkish sandwich\": \"The user mentioned that they had a Turkish sandwich with a unique jam inside.\"\n}"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 133,
    "completion_tokens": 123,
    "total_tokens": 256
  }
}


In [43]:
ast.literal_eval(response['choices'][0]["message"]["content"])

{'local beer': 'The user mentioned that they ordered a local beer, but did not provide any further details about it.',
 'hamburger': 'The user mentioned that they ordered a hamburger and thought it was okay.',
 'regular burger': 'The user mentioned that they had a regular burger and liked it better than the sandwich.',
 'sandwich': 'The user mentioned that they had a sandwich with a unique jam inside, but did not provide any further details about it.',
 'Turkish sandwich': 'The user mentioned that they had a Turkish sandwich with a unique jam inside.'}

Next, extract just the content. You will notice it is a string with ```\n``` and ```\```. You can turn this into a Python object using ```ast.literal_eval```. This turns a string of a dictionary into an actual dictionary.

In the cell below, extract the API's response content and convert it into a Python dictionary using ```ast.literal_eval```. (You may need to navigate the ```response``` dictionary a bit to find what you want)

In [44]:
response_as_dict = ast.literal_eval(response['choices'][0]["message"]["content"])

print(f' Original review: {sample_review}')
pprint.PrettyPrinter().pprint(response_as_dict)
print('\n')

 Original review: I ordered a local beer and a hamburger, but it was okay. The regular burger was better than the sandwich. The Turkish sandwich had a unique jam inside.
{'Turkish sandwich': 'The user mentioned that they had a Turkish sandwich with '
                     'a unique jam inside.',
 'hamburger': 'The user mentioned that they ordered a hamburger and thought it '
              'was okay.',
 'local beer': 'The user mentioned that they ordered a local beer, but did not '
               'provide any further details about it.',
 'regular burger': 'The user mentioned that they had a regular burger and '
                   'liked it better than the sandwich.',
 'sandwich': 'The user mentioned that they had a sandwich with a unique jam '
             'inside, but did not provide any further details about it.'}




In [45]:
check('4.1.5', response_as_dict)

## Correct! 🎉

Does this summary make sense?

Next, let's try doing this for multiple reviews.

In the cell below, extend the prompt to include all the reviews in the list ```reviews```. You have the choice of the following:

1. Combine all the reviews into one giant string and combine that with your original prompt

2. Run the prompt once for each review, and then merge the dictionaries into one

3. Do the reviews in chunks (a mix between one and two).

A few hints: (1) If you are running into token limits from OpenAI, you will need to find a way to divide your prompt into smaller bits and combinin them after. (2) Always be as explicit with the API as possible! (Tell it what you are putting in as input, and what you expect to receive as output).



In [48]:
";".join(local_df.loc[:,'text'].to_list())

'the food is really good, I had a great experience. they have a big variety.;High prices for not so big portions! But I think it is the only decent food available on the National Mall area without walking a few miles away from there.;The gelato looked "old" and ended up having coffee.;(Translated by Google) The fast food restaurants under the National Palace of Art are not cheap, and there are many choices. They are all Western food. The dining environment is very good and the art atmosphere is strong.\n\n(Original)\n国家艺术宫下的快餐厅，不便宜，选择多，都是西餐，就餐环境非常好，艺术气息浓厚。;(Translated by Google) The food is barely edible and the price is high. The rice was uncooked and the pasta was blown, making it difficult to eat. The seafood pasta contained 4 mussels, but only two of them were shells. It is a strong light. Never go.\n\n(Original)\n음식은 거의 먹을 수 없는 수준이며 가격은 비쌉니다. 밥은 생쌀에다가 파스타는 불어서 먹기 힘이 들었습니다. 해물 파스타에 홍합이 4개 들었는데 그나마 두개는 껍질뿐이었습니다. 강력 비추입니다. 절대 가지 마세요.;Rip off!!!\nI have dreads and I just needed a quic

In [66]:
# Run these lines
system_prompt = 'You are an restaurant review writer whose job it is to help \
               people find good menu items. You will return responses is json \
                dictionary format only'
local_df.loc[:,'text']=local_df.loc[:,'text'].str.replace('"";','')
reviews = ";".join(local_df.loc[:,'text'].to_list())
prompt =  f'The following text is a list of restaurant reviews seperated by ; . \
        For each unique menu item mentioned, give a summary of \
        the user reviews for that menu item. \
        Return the output in the form of a json dictionary, \
        where the keys are unique menu items \
        and the values are the list of all summarys for that item. : "{reviews}"'

In [68]:
response_dict_list  = []
for i in range(0,1000,300):
  print(i,i+300)
  reviews = " ".join(local_df.loc[i:i+100,'text'].to_list())
  system_prompt = f'You are an restaurant review writer whose job it is to help \
               people find good menu items. You will return responses is json \
                dictionary format only'

  prompt =  f'The following text is a list of restaurant reviews seperated by . \
        For each unique menu item mentioned, give a summary of \
        the user reviews for that menu item. \
        Return the output in the form of a json dictionary, \
        where the keys are unique menu items \
        and the values are the list of all summarys for that item. : "{reviews}"'
        # Generate response
  messages=[{'role': 'system', 'content': f"{system_prompt}"}, {'role': 'user', 'content': f"{prompt}"}]
  response = openai.ChatCompletion.create(model="gpt-3.5-turbo",messages=messages)
  response_as_dict = ast.literal_eval(response['choices'][0]["message"]["content"])
  response_dict_list.append(response_as_dict)





0 300
300 600
600 900
900 1200


In [71]:
final_dict = {}

for dictionary in response_dict_list:
    for key, value in dictionary.items():
        if key in final_dict:
            final_dict[key].append(value)
        else:
            final_dict[key] = value

pprint.PrettyPrinter().pprint(final_dict)

{'DMV Entrance': ["Good customer service You guys are awesome! I'm coming "
                  'tomorrow again just to thank each and every one of you. :-)',
                  'I never have to wait longer than 20 mins to be in and out. '
                  'Quick courteous and helpful DMV Entrance on Kenilworth Ave '
                  'its a one-way street you have to drive slowly or you will '
                  'very easily. Building Entrance is after Mobil Gas station '
                  '⛽. pearl parking size is 22 X 8 feet at this location. They '
                  'have 2 spots for tests. Community Dinner Tuesday 5:30 to '
                  '6:30 I Love it Nice Love it'],
 'Exxon': ['Jamie and the team are top notch professionals who proactively '
           'address any and all car issues you may encounter.',
           "Couldn't recommend them more Jamie was great the manager when I "
           'was unloading the Volvo. He was able to assist.',
           'I pulled into Exxon Fri

In [50]:
prompt

'The following text is a list of restaurant reviews seperated by ; .         For each unique menu item mentioned, give a summary of         the user reviews for that menu item.         Return the output in the form of a json dictionary,         where the keys are unique menu items         and the values are the list of all summarys for that item. : "the food is really good, I had a great experience. they have a big variety.;High prices for not so big portions! But I think it is the only decent food available on the National Mall area without walking a few miles away from there.;The gelato looked "old" and ended up having coffee.;(Translated by Google) The fast food restaurants under the National Palace of Art are not cheap, and there are many choices. They are all Western food. The dining environment is very good and the art atmosphere is strong.\n\n(Original)\n国家艺术宫下的快餐厅，不便宜，选择多，都是西餐，就餐环境非常好，艺术气息浓厚。;(Translated by Google) The food is barely edible and the price is high. The rice was u

You should have a much longer dictionary, with many more menu items now!

You may have noticed if you called the API multiple times that each call yielded a different result. This is due to the inherent randomness in the model's predictions.

Let's investigate the sources of randomness a little bit here.

In the cell below, generate a second summary of the reviews and store it in the variable ```response_as_dict2```.

This time, add in as a parameter to ```openai.ChatCompletion.create``` the variable ```temperature```. In general, ```temperature``` should be between 0 and 2, with 0 being the least amount of randomness and 2 being the greatest amount of randomness. Choose a temperature of 1.1 and see what the output looks like. (Hint: If the output cannot be interpreted as a dictionary, try running it again, or change the prompt!)



In [73]:
response_dict_list  = []
for i in range(0,1000,300):
  print(i,i+300)
  reviews = " ".join(local_df.loc[i:i+100,'text'].to_list())
  system_prompt = f'You are an restaurant review writer whose job it is to help \
               people find good menu items. You will return responses is json \
                dictionary format only'

  prompt =  f'The following text is a list of restaurant reviews seperated by . \
        For each unique menu item mentioned, give a summary of \
        the user reviews for that menu item. \
        Return the output in the form of a json dictionary, \
        where the keys are unique menu items \
        and the values are the list of all summarys for that item. : "{reviews}"'
        # Generate response
  messages=[{'role': 'system', 'content': f"{system_prompt}"}, {'role': 'user', 'content': f"{prompt}"}]
  response = openai.ChatCompletion.create(model="gpt-3.5-turbo",messages=messages, temperature =1.1)
  response_as_dict = ast.literal_eval(response['choices'][0]["message"]["content"])
  response_dict_list.append(response_as_dict)


0 300
300 600
600 900
900 1200


In [74]:
final_dict = {}

for dictionary in response_dict_list:
    for key, value in dictionary.items():
        if key in final_dict:
            final_dict[key].append(value)
        else:
            final_dict[key] = value

response_as_dict2=final_dict
pprint.PrettyPrinter().pprint(response_as_dict2)

{'Conte’s': ['What initially drove me to Conte’s was the simple fact they were '
             'open during their posted hours. Easy enough, right? Well, I’ve '
             'been going ever since and recommend them as often as able.\n'
             '\n'
             'This latest visit was for a Spring tune-up and the funny thing '
             'is how I seemingly brought in a Corolla and got back a Supra. '
             'Clearly all maintenance boxes were checked off but what shocked '
             'me was how clean they got it. Riding a white bike shows just '
             'about everything and this isn’t just the frame but the '
             'handlebars too. SPOTLESS This said, it’s well worth the cost and '
             'time away from your personal if you’re on the fence…\n'
             '\n'
             '“Conte’s Logan Circle”:\n'
             '1. Staff is welcoming and knowledgeable.\n'
             '2. Service prices are competitive.\n'
             '3. Overall experience is un

What do you notice about the output? While each run is different, you may see the following: (1) different menu items, (2) different styles of description for each menu item, or (3) a different format alltogether.

In practice, it is good to get a sample of the types of outputs being produced. Another parameter to use is ```n```, which just controls the number of samples you want ChatGPT to produce.

In the cell below, rather than changing temperature, just set ```n=2```.

In [75]:
response_dict_list  = []
for i in range(0,1000,300):
  print(i,i+300)
  reviews = " ".join(local_df.loc[i:i+100,'text'].to_list())
  system_prompt = f'You are an restaurant review writer whose job it is to help \
               people find good menu items. You will return responses is json \
                dictionary format only'

  prompt =  f'The following text is a list of restaurant reviews seperated by . \
        For each unique menu item mentioned, give a summary of \
        the user reviews for that menu item. \
        Return the output in the form of a json dictionary, \
        where the keys are unique menu items \
        and the values are the list of all summarys for that item. : "{reviews}"'
        # Generate response
  messages=[{'role': 'system', 'content': f"{system_prompt}"}, {'role': 'user', 'content': f"{prompt}"}]
  response = openai.ChatCompletion.create(model="gpt-3.5-turbo",messages=messages, temperature =0,n=2)
  response_as_dict = ast.literal_eval(response['choices'][0]["message"]["content"])
  response_dict_list.append(response_as_dict)

0 300
300 600
600 900
900 1200


In [76]:
final_dict = {}

for dictionary in response_dict_list:
    for key, value in dictionary.items():
        if key in final_dict:
            final_dict[key].append(value)
        else:
            final_dict[key] = value

response_as_dict2=final_dict
pprint.PrettyPrinter().pprint(response_as_dict2)

{'Conte’s': ['What initially drove me to Conte’s was the simple fact they were '
             'open during their posted hours. Easy enough, right? Well, I’ve '
             'been going ever since and recommend them as often as able. This '
             'latest visit was for a Spring tune-up and the funny thing is how '
             'I seemingly brought in a Corolla and got back a Supra. Clearly '
             'all maintenance boxes were checked off but what shocked me was '
             'how clean they got it. Riding a white bike shows just about '
             'everything and this isn’t just the frame but the handlebars too. '
             'SPOTLESS This said, it’s well worth the cost and time away from '
             'your personal if you’re on the fence… “Conte’s Logan Circle”: 1. '
             'Staff is welcoming and knowledgeable. 2. Service prices are '
             'competitive. 3. Overall experience is unmatched. There are a few '
             'bike shops in the area, each wit

In [ ]:
# Generate response again but with n=2
response = ...

# Store as a dictionary
response_run1 = ...
response_run2 = ...

Take a look at a specific menu item between the first and the second call. What kinds of differences do you notice? Are the menu items picked up by the API the same?

Feel free to post your findings on #deep-learning-questions-and-debugging to see if others have gotten similar differences!

This next part is pretty open-ended, so feel free to take some creative liberties. We want to use ChatGPT to filter its own outputs. What kind of prompt would you use to combine the two generated responses into a better, third output?

In [ ]:
combine_system = ...
combine_prompt = ...

response = ...
combined_response = ...

In practice, the strategy of using the large language model to refine itself is used quite often! While there are many possible ways to get the prompts to do what you want, here are a few guidelines provided by OpenAI: [cookbook](https://github.com/openai/openai-cookbook/blob/main/techniques_to_improve_reliability.md)



## Open-Ended Exploration (Optional)

We hope this tutorial was helpful in understanding the ins and outs of OpenAI's ChatGPT API.

There are definitely many more things we can do with this dataset, but we would love to see what you can think of.

In the remaining part of this assignment, feel free to come up with a task and post your findings on Slack under #deep-learning-questions-and-debugging.

Here are a few ideas:

1. Look for menu items across restaurants and find the best restaurant for a specific menu item.

2. Compare and contrast two businesses given their reviews.

3. Find the most controversial dish!

## Final Step

Before submitting this assignment, remember to remove your OpenAI API key from the beginning of this assignment 😀.